In [1]:
import pandas as pd
import numpy as np
#from tqdm.notebook import tqdm
#import plotly.graph_objects as go
import re
import os

### 20년도 매출 데이터 확인

In [2]:
file_names = list(os.walk('../datas/huge/project_visualization'))[0][2]
csvs = [pd.read_csv(os.path.join('../datas/huge/project_visualization', file_name), encoding='cp949') for file_name in file_names]
df_sales_2020 = pd.concat(csvs)
df_sales_2019 = pd.concat([
    pd.read_csv('../datas/huge/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='cp949'),
])

In [3]:
df_sales_2020

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO
0,0,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455,...,345,8752,7776,611,3828,4356,3428,2614,1690,19
1,1,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161,...,0,592,694,145,368,240,368,96,69,1
2,2,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866,...,241,622,508,0,134,245,275,314,162,3
3,3,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306,...,33,1281,2014,5,1361,1116,331,325,157,3
4,4,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129,...,1205,10426,14742,234,4316,7089,6188,4351,2989,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2020,4,A,골목상권,1000052,녹사평대로32길,CS300024,운동/경기용품,856662,...,0,48,0,0,0,0,48,0,0,3
996,996,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100001,한식음식점,418790754,...,915,5371,5350,245,5771,2713,1175,605,213,7
997,997,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100002,중식음식점,11078526,...,94,274,174,0,210,124,28,86,0,1
998,998,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100003,일식음식점,90936583,...,252,276,73,0,68,58,121,68,35,1


In [4]:
df_sales_2019

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2019,1,A,골목상권,1000985,암사길,CS200003,예술학원,20061598,125,...,0,93,32,0,0,32,93,0,0,3
1,2019,1,A,골목상권,1000267,돌곶이로8길,CS200037,노래방,510636,21,...,8,4,17,0,0,0,9,4,8,1
2,2019,1,R,전통시장,1001445,영도시장,CS300027,섬유제품,4038928,33,...,0,17,0,0,0,17,0,0,0,2
3,2019,1,A,골목상권,1000789,난곡로24길,CS300017,시계및귀금속,5085873,10,...,0,5,5,0,0,0,5,0,5,1
4,2019,1,A,골목상권,1000930,언주로81길,CS200001,일반교습학원,527027,53,...,53,53,0,0,0,0,0,53,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135573,2019,4,R,전통시장,1001469,양재시장,CS300011,일반의류,14282341,48,...,0,0,48,0,0,48,0,0,0,2
135574,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,...,228,12935,9718,469,6166,6432,4792,3474,1319,2
135575,2019,4,A,골목상권,1000441,세무서2길,CS300002,편의점,518371562,81848,...,17654,57240,24608,2813,30887,15345,14338,12869,5596,2
135576,2019,4,D,발달상권,1001044,양재역_3,CS200001,일반교습학원,15509601,56,...,6,25,31,0,0,12,44,0,0,6


In [5]:
df_sales_2020.drop_duplicates()

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO
0,0,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455,...,345,8752,7776,611,3828,4356,3428,2614,1690,19
1,1,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161,...,0,592,694,145,368,240,368,96,69,1
2,2,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866,...,241,622,508,0,134,245,275,314,162,3
3,3,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306,...,33,1281,2014,5,1361,1116,331,325,157,3
4,4,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129,...,1205,10426,14742,234,4316,7089,6188,4351,2989,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2020,4,A,골목상권,1000052,녹사평대로32길,CS300024,운동/경기용품,856662,...,0,48,0,0,0,0,48,0,0,3
996,996,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100001,한식음식점,418790754,...,915,5371,5350,245,5771,2713,1175,605,213,7
997,997,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100002,중식음식점,11078526,...,94,274,174,0,210,124,28,86,0,1
998,998,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100003,일식음식점,90936583,...,252,276,73,0,68,58,121,68,35,1


In [6]:
df_sales_2020[df_sales_2020['TRDAR_SE_CD_NM'] == '전통시장']

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO
19,19,2020,2,R,전통시장,1001352,번동북부골목시장,CS200028,미용실,90507273,...,75,1862,1525,28,382,472,1082,983,440,8
20,20,2020,2,R,전통시장,1001352,번동북부골목시장,CS200029,네일숍,521733,...,0,5,10,0,5,5,0,5,0,0
95,95,2020,2,R,전통시장,1001352,번동북부골목시장,CS200031,세탁소,10754472,...,43,298,256,21,86,192,106,149,0,2
117,117,2020,4,R,전통시장,1001460,관악신사시장,CS300018,의약품,271160578,...,0,10719,13341,197,2771,4362,5964,5889,4877,4
463,463,2020,4,R,전통시장,1001475,강남개포시장,CS100007,치킨전문점,87505216,...,1098,1926,1308,195,438,348,937,1027,291,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,2020,3,R,전통시장,1001490,길동복조리시장,CS300022,화장품,1010676465,...,118,15494,28948,0,1454,4155,4614,14528,19690,6
887,887,2020,3,R,전통시장,1001490,길동복조리시장,CS300027,섬유제품,170000001,...,25,934,2081,0,192,279,562,805,1177,5
888,888,2020,3,R,전통시장,1001490,길동복조리시장,CS300028,화초,60772803,...,245,2104,665,35,1474,735,280,210,35,4
889,889,2020,3,R,전통시장,1001490,길동복조리시장,CS300029,애완동물,27813477,...,0,379,1105,0,180,174,321,494,315,2


In [7]:
df_sales_2019[df_sales_2019['상권_구분_코드_명'] == '전통시장']

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2019,1,R,전통시장,1001445,영도시장,CS300027,섬유제품,4038928,33,...,0,17,0,0,0,17,0,0,0,2
6,2019,1,R,전통시장,1001483,암사종합시장,CS300008,수산물판매,1712543938,108896,...,511,40273,68502,116,2309,12234,21650,33132,39332,9
14,2019,1,R,전통시장,1001316,서울약령시장,CS200008,한의원,3329153658,38642,...,26,14251,24366,10,1441,3620,6465,7383,19683,82
15,2019,1,R,전통시장,1001314,경동시장,CS300007,육류판매,4587060783,84286,...,5,31843,51739,30,1370,6433,11198,20193,44356,30
28,2019,1,R,전통시장,1001323,답십리시장,CS200028,미용실,100328154,2550,...,0,1470,1080,50,483,623,596,604,194,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135547,2019,4,R,전통시장,1001317,청량리농수산물시장,CS300009,청과상,86437896,1553,...,0,1553,0,0,0,518,517,0,518,15
135565,2019,4,R,전통시장,1001416,공항시장,CS100010,커피-음료,15675910,2016,...,182,950,907,15,777,392,358,262,54,1
135566,2019,4,R,전통시장,1001471,영동전통시장,CS300010,반찬가게,4773539895,118316,...,1871,41841,74436,661,11780,25687,27632,27036,23478,12
135571,2019,4,R,전통시장,1001427,현대시장,CS300019,의료기기,32120476,757,...,0,344,413,0,0,48,122,233,354,3


In [8]:
df_sales_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132321 entries, 0 to 999
Data columns (total 81 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Unnamed: 0                 132321 non-null  int64 
 1   STDR_YY_CD                 132321 non-null  int64 
 2   STDR_QU_CD                 132321 non-null  int64 
 3   TRDAR_SE_CD                132321 non-null  object
 4   TRDAR_SE_CD_NM             132321 non-null  object
 5   TRDAR_CD                   132321 non-null  int64 
 6   TRDAR_CD_NM                132321 non-null  object
 7   SVC_INDUTY_CD              132321 non-null  object
 8   SVC_INDUTY_CD_NM           132321 non-null  object
 9   THSMON_SELNG_AMT           132321 non-null  int64 
 10  THSMON_SELNG_CO            132321 non-null  int64 
 11  MDWK_SELNG_RATE            132321 non-null  int64 
 12  WKEND_SELNG_RATE           132321 non-null  int64 
 13  MON_SELNG_RATE             132321 non-null  int

In [9]:
df_sales_2020[df_sales_2020['TRDAR_CD_NM'].str.contains('방림')]

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO


In [10]:
df_sales_2020 = df_sales_2020.drop('Unnamed: 0', axis=1)

In [11]:
df_sales_2020.columns = df_sales_2019.columns

In [12]:
df_sales_2020.to_csv('../datas/huge/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='cp949')

### 시장주소, 생필품 데이터 확인

#### 시장주소 데이터 확인
- 창동신창시장 -> 신창시장으로 통일
- 송화시장 -> 송화골목시장으로 통일
- 공릉동 도깨비시장 -> 공릉동도깨비시장	
- 시장주소에 있는 대림시장은 삭제

In [13]:
market_mapping = {
    '창동신창시장': '신창시장',
    '송화시장': '송화골목시장',
    '공릉동 도깨비시장': '공릉동도깨비시장',
    '신원시장(신림1동)': '신원시장',
    '수유전통시장(구)수유골목시장': '수유전통시장',
    '원당종합시장': '인헌시장',
    '관악신사시장(신림4동)': '관악신사시장'
}

In [14]:
df_customer = pd.read_csv('../datas/huge/df_customer.csv', encoding='cp949')
df_market_address = pd.read_csv('../datas/huge/df_market_address.csv', encoding='cp949')
df_market_address2 = pd.read_csv('../datas/huge/df_market_address2.csv', encoding='cp949')
df_necessity = pd.read_csv('../datas/huge/df_necessity.csv', encoding='cp949')
df_necessity2 = pd.read_csv('../datas/huge/df_necessity2.csv', encoding='cp949')
df_sales = pd.read_csv('../datas/huge/df_sales.csv', encoding='cp949')

In [15]:
# df_market_address에서 market_mapping을 적용해 replace한 다음, 그대로 데이터 저장
df_market_address['시장이름'] = df_market_address['시장이름'].replace(market_mapping)
df_market_address2['시장이름'] = df_market_address2['시장이름'].replace(market_mapping)

In [16]:
df_market_address = df_market_address.drop(['Unnamed: 0', '주소명', '자치구'], axis=1)

In [17]:
df_market_address2 = df_market_address2.drop('Unnamed: 0', axis=1)

In [18]:
pd.concat([df_market_address, df_market_address2]).drop_duplicates(keep=False)

,시장이름,위도,경도
59,대림시장,37.491372,126.907663


In [19]:
df_market_address[df_market_address['시장이름']=='대림시장']

,시장이름,위도,경도
58,대림시장,37.586763,126.91761


In [20]:
# 59번 인덱스 대림시장 삭제해야 함
df_market_address2[df_market_address2['시장이름']=='대림시장']

,시장이름,위도,경도
58,대림시장,37.586763,126.917610
59,대림시장,37.491372,126.907663


In [21]:
df_market_address2 = df_market_address2.drop(59)

In [22]:
# 시장주소 dataframe 정리 완료
pd.concat([df_market_address, df_market_address2]).drop_duplicates(keep=False)

,시장이름,위도,경도


In [23]:
df_market_add = pd.concat([df_market_address, df_market_address2]).drop_duplicates()

In [24]:
# df_market_add.

In [25]:
#df_market_address['시장이름'].unique()

In [26]:
# df_market_address2['시장이름'].unique()

#### 생필품 데이터 확인
- 일련번호 drop
- 년도, 분기 컬럼 분리
- 품목 이름 -> 품목이름
- 자치구 -> 자치구이름

In [27]:
# 생필품 데이터에서 년도-분기 매칭 완료
condlist = []
choicelist_year = []
choicelist_quarter = []

for year_month in df_necessity['년도-월'].unique():
    year, month = year_month.split('-')
    quarter = ''
    if month in {'01', '02', '03'}:
        quarter = 1
    elif month in {'04', '05', '06'}:
        quarter = 2
    elif month in {'07', '08', '09'}:
        quarter = 3
    elif month in {'10', '11', '12'}:
        quarter = 4
    condlist.append(df_necessity['년도-월'] == year_month)
    choicelist_year.append(year)
    choicelist_quarter.append(quarter)

df_necessity['년도'] = np.select(condlist, choicelist_year)
df_necessity['분기'] = np.select(condlist, choicelist_quarter)

In [28]:
df_necessity['년도'] = df_necessity['년도'].astype('int64')
df_necessity['분기'] = df_necessity['분기'].astype('int64')

In [29]:
df_necessity = df_necessity.drop(['Unnamed: 0', '비고', '업종', '년도-분기'], axis=1).rename(columns={'품목 이름': '품목이름', '자치구': '자치구이름'})

In [30]:
df_necessity2 = df_necessity2.drop(['Unnamed: 0'], axis=1).rename(columns={'품목 이름': '품목이름', '자치구명': '자치구이름'})

In [31]:
df_necessity['시장이름'].unique()

array(['통인시장', '방배종합시장', '방학동도깨비시장', '장위골목시장', '신창시장', '마포농수산물시장', '동원시장',
       '광장시장', '남성시장', '금남시장', '뚝도시장', '후암시장', '청량리종합시장', '남문시장', '남구로시장',
       '신영시장', '고척근린시장', '용문시장', '대림시장', '영등포전통시장', '망원시장', '영천시장',
       '인왕시장', '신원시장(신림1동)', '자양골목시장', '수유재래시장', '경동시장', '우림골목시장', '송화시장',
       '대림중앙시장', '목3동시장', '공릉동 도깨비시장', '돈암제일시장', '원당종합시장', '남대문시장',
       '청담삼익시장', '현대시장', '둔촌역전통시장', '서울중앙시장', '노룬산골목시장', '화곡본동시장', '방이시장',
       '대조시장', '도곡시장', '마천중앙시장', '관악신사시장(신림4동)', '암사종합시장', '상계중앙시장',
       '숭인시장', '방림시장', '구로시장', '영동전통시장'], dtype=object)

In [32]:
df_necessity['시장이름'] = df_necessity['시장이름'].replace(market_mapping)

In [33]:
df_necessity2['시장이름'] = df_necessity2['시장이름'].replace(market_mapping)

#### df_necessity, df_necessity2 순
- 우림골목시장 <- 우림시장
- 도곡시장 살림
- 방림시장 죽임
- 남구로시장 현용 데이터만 삭제 (번호-시장이름 오류를 고친 것 때문에 나오는 것일 듯)
- 신영시장, 신영시장
- 목3동시장, 목3동시장

In [34]:
market_mapping2 = {
    '방배종합시장': "남부종합시장", 
    '남성시장': "사당시장", 
    '영동시장': "영동전통시장", 
#     '장위골목시장': "장위전통시장", 
    '장위전통시장': "장위골목시장", 
#     '용문시장': "용문전통시장",
    '용문전통시장': "용문시장",
    '신원시장(신림1동)': "신원시장", 
    '수유재래시장': "수유전통시장", 
    '원당종합시장': "인헌시장",
    '마천중앙시장': "마천시장", 
    '관악신사시장(신림4동)': "관악신사시장",
    '우림시장': '우림골목시장',
}
# necessity_dead = ['방림시장']
# necessity2_dead = ['남구로시장']

# test_dead = ['도곡시장', '구로시장']

In [35]:
df_necessity['시장이름'] = df_necessity['시장이름'].replace(market_mapping2)

In [36]:
df_necessity2['시장이름'] = df_necessity2['시장이름'].replace(market_mapping2)

In [37]:

df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='방림시장'].index)
df_necessity2 = df_necessity2.drop(df_necessity2[df_necessity2['시장이름']=='남구로시장'].index)

df_necessity2 = df_necessity2.dropna(subset=['품목이름'])

# for duplicate check(나중에 살려야하는 자료)
#df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='구로시장'].index)
#df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='남구로시장'].index)
#df_necessity2 = df_necessity2.drop(df_necessity2[df_necessity2['시장이름']=='구로시장'].index)
#df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='도곡시장'].index)

In [38]:
pd.concat([df_necessity, df_necessity2]).drop_duplicates(keep=False)['시장이름'].unique()

array(['남구로시장', '도곡시장', '구로시장'], dtype=object)

In [39]:
pd.concat([df_necessity, df_necessity2]).drop_duplicates(keep=False)

,일련번호,시장이름,품목이름,실판매규격,가격(원),년도-월,자치구이름,년도,분기
510,1630466,남구로시장,사과,1개 (300g),1000,2020-12,구로구,2020,4
511,1623936,남구로시장,사과,1개 300g,1000,2020-12,구로구,2020,4
516,1629271,남구로시장,애호박,1개 (300g),1000,2020-12,구로구,2020,4
665,1630479,남구로시장,조기,1마리[20cm],2000,2020-12,구로구,2020,4
667,1622512,남구로시장,쇠고기,600g,33000,2020-12,구로구,2020,4
...,...,...,...,...,...,...,...,...,...
160728,449830,구로시장,달걀,10개,1500,2013-05,구로구,2013,2
160729,449831,구로시장,"조기(냉동,수입산)",1마리(20cm),1500,2013-05,구로구,2013,2
160730,449832,구로시장,"명태(러시아,냉동)",1마리(42cm),1700,2013-05,구로구,2013,2
160731,449833,구로시장,"오징어(생물,국산)",1마리,2500,2013-05,구로구,2013,2


In [40]:
# 최종 생필품 데이터
df_necessities = pd.concat([df_necessity, df_necessity2]).drop_duplicates()
df_necessities

,일련번호,시장이름,품목이름,실판매규격,가격(원),년도-월,자치구이름,년도,분기
0,1621162,통인시장,양파(1.5kg망),1kg,2000,2020-12,종로구,2020,4
1,1628949,남부종합시장,호박,1개268g,1500,2020-12,서초구,2020,4
2,1622882,통인시장,양파(1.5kg망),1kg,2000,2020-12,종로구,2020,4
3,1622883,통인시장,상추(100g),400g,3000,2020-12,종로구,2020,4
4,1622885,통인시장,호박(인큐베이터),1개,1000,2020-12,종로구,2020,4
...,...,...,...,...,...,...,...,...,...
160728,449830,구로시장,달걀,10개,1500,2013-05,구로구,2013,2
160729,449831,구로시장,"조기(냉동,수입산)",1마리(20cm),1500,2013-05,구로구,2013,2
160730,449832,구로시장,"명태(러시아,냉동)",1마리(42cm),1700,2013-05,구로구,2013,2
160731,449833,구로시장,"오징어(생물,국산)",1마리,2500,2013-05,구로구,2013,2


In [41]:
#df_necessity.info()

In [42]:
#df_necessity2.info()

In [43]:
df_necessity.groupby(['시장이름', '품목이름', '실판매규격']).count()

일련번호  가격(원)  년도-월  자치구이름  년도  분기
시장이름 품목이름       실판매규격                                       
경동시장 고등어(냉동,국산) 1마리(20cm)      2      2     2      2   2   2
     고등어(생물,국산) 1ㅁ리(25cm)      1      1     1      1   1   1
                1마리((25cm)     1      1     1      1   1   1
                1마리(20cm)      5      5     5      5   5   5
                1마리(258g)      1      1     1      1   1   1
...                          ...    ...   ...    ...  ..  ..
후암시장 조기(생물,수입산) 1마리 35cm       1      1     1      1   1   1
                1마리20cm)       6      6     6      6   6   6
     조기(중국산,생물) 1마리20          4      4     4      4   4   4
                1마리25cm)       3      3     3      3   3   3
     호박         1개            23     23    23     23  23  23

[9036 rows x 6 columns]

### 유동인구 데이터 확인

In [44]:
market_mapping3 = {
    '대림골목시장': '대림시장',
}

In [45]:
df_customer = pd.read_csv('../datas/huge/df_customer.csv', encoding='cp949')

In [46]:
df_customer = df_customer.drop('Unnamed: 0', axis=1)

In [47]:
df_customer['시장이름'] = df_customer['시장이름'].replace(market_mapping3)

In [48]:
for i in range(len(df_customer['시장이름'])):
    if '대림골목시장' in df_customer['시장이름'][i] :
        print('try again')

In [49]:
# 최종 유동인구 데이터
df_customer

,년도,분기,시장이름,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,...,여성연령대_60_이상_토요일시간대_3_생활인구_수,여성연령대_60_이상_토요일시간대_4_생활인구_수,여성연령대_60_이상_토요일시간대_5_생활인구_수,여성연령대_60_이상_토요일시간대_6_생활인구_수,여성연령대_60_이상_일요일시간대_1_생활인구_수,여성연령대_60_이상_일요일시간대_2_생활인구_수,여성연령대_60_이상_일요일시간대_3_생활인구_수,여성연령대_60_이상_일요일시간대_4_생활인구_수,여성연령대_60_이상_일요일시간대_5_생활인구_수,여성연령대_60_이상_일요일시간대_6_생활인구_수
0,2020,4,길동복조리시장,873376,396767,476609,144022,94019,114971,136136,...,2478,2523,3412,2641,5460,4288,2493,2562,3545,2713
1,2020,4,둔촌역전통시장,252938,122996,129942,36836,31794,34053,40459,...,786,821,1165,657,1258,1082,780,830,1103,644
2,2020,4,성내골목시장,641789,288503,353286,100176,73678,99616,100460,...,1936,2199,3282,1741,3131,2678,1903,2250,3129,1694
3,2020,4,천호시장,91897,41183,50716,7332,16546,16344,14186,...,313,362,406,190,283,281,281,333,374,177
4,2020,4,동서울시장,66313,29932,36381,6265,9898,11419,9704,...,210,227,256,176,344,289,201,209,251,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,2014,1,충신시장,38363,24613,13750,236,5863,10314,10363,...,30,19,10,0,0,0,4,3,0,0
6279,2014,1,종로신진시장,190225,118296,71928,394,27801,48260,45866,...,283,226,91,3,0,1,12,23,14,2
6280,2014,1,동대문종합시장,496529,277937,218593,10582,98155,135325,112571,...,684,562,383,82,1,87,203,217,198,67
6281,2014,1,광장시장,238439,157317,81122,1059,22979,49399,59886,...,454,452,243,15,0,4,77,117,68,2


### 매출 데이터 확인

In [50]:
df_sales = df_sales.rename(columns={'기준_년_코드': '년도', '기준_분기_코드': '분기'})

In [51]:
df_necessities['년도'] = df_necessities['년도'].astype('int64')
df_necessities['분기'] = df_necessities['분기'].astype('int64')

In [52]:
# 210706김기찬에서 코드 가져옴
# item_names의 결과에 업종을 수작업으로 매핑
# category = ['수산물판매', '육류판매', '미곡판매', '청과상']
item_to_category = {
    '애호박': '청과상', 
    '오징어': '수산물판매', 
    '상추': '청과상', 
    '사과': '청과상', 
    '오이': '청과상', 
    '달걀': '육류판매', 
    '돼지고기': '육류판매', 
    '냉동참조기': '수산물판매', 
    '양파': '청과상', 
    '조기': '수산물판매', 
    '동태': '수산물판매', 
    '호박': '청과상', 
    '쇠고기': '육류판매', 
    '고등어': '수산물판매', 
    '닭고기': '육류판매', 
    '명태': '수산물판매', 
    '무': '청과상', 
    '배': '청과상', 
    '배추': '청과상',
    '삼겹살': '육류판매',
    '갈치': '수산물판매',
}
# 품목 이름, 업종 매칭 완료
condlist = []
choicelist = []

for item, category in item_to_category.items():
    condlist.append(df_necessities['품목이름'].str.contains('^' + item))
    choicelist.append(category)

df_necessities['업종'] = np.select(condlist, choicelist)

In [53]:
df_sales = df_sales.drop(['Unnamed: 0', 'Unnamed: 0.1', '년도-분기'], axis=1)

In [54]:
market_mapping4 = {
    '관악신사시장(신림4동)' : '관악신사시장', 
    '마천중앙시장' : '마천시장', 
    '송화시장' : '송화골목시장', 
    '수유재래시장' : '수유전통시장', 
    '신원시장(신림1동)' : '신원시장'
}

In [55]:
df_sales['시장이름'] = df_sales['시장이름'].replace(market_mapping4)

In [56]:
set(df_necessities['시장이름'].unique()) - set(df_sales['시장이름'].unique())

set()

In [57]:
df_sales[df_sales['시장이름'].str.contains('남부')]['시장이름'].unique()

array(['남부골목시장', '남부종합시장'], dtype=object)

In [58]:
df_sales[df_sales['시장이름']=='남부종합시장']['업종'].unique()

array(['한식음식점', '분식전문점', '가전제품수리', '반찬가게', '화초', '철물점', '일반의류', '시계및귀금속',
       '전자상거래업', '커피-음료', '청과상', '문구', '의약품', '가구', '미용실', '화장품'],
      dtype=object)

In [59]:
df_sales[(df_sales['시장이름']=='남부종합시장') & (df_sales['업종']=='청과상')]

,년도,분기,시장이름,업종,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
55141,2017,1,남부종합시장,청과상,19784932,122,100,0,31,26,...,0,24,8,0,0,0,0,32,0,1
59403,2017,2,남부종합시장,청과상,18700015,123,100,0,28,20,...,0,32,8,0,0,0,8,24,8,1
65370,2017,3,남부종합시장,청과상,6118382,32,100,0,0,50,...,0,16,0,0,0,0,0,16,0,0
74350,2016,1,남부종합시장,청과상,27433902,162,95,5,11,42,...,16,40,39,0,0,0,39,32,8,1
82521,2016,2,남부종합시장,청과상,30034620,181,100,0,20,38,...,8,58,44,0,0,0,32,48,22,1
86016,2016,3,남부종합시장,청과상,16318390,122,100,0,46,21,...,0,24,48,0,0,16,32,24,0,1
92134,2016,4,남부종합시장,청과상,24631761,122,100,0,5,51,...,8,16,64,0,0,8,16,56,0,1


In [60]:
df_necessities[df_necessities['시장이름']=='남부종합시장']['년도'].unique()

array([2020], dtype=int64)

In [61]:
# 남부종합시장 데이터는 생필품데이터에는 2020년밖에 없음
# 따라서 남부종합시장은 제외
df_necessities = df_necessities[df_necessities['시장이름']!='남부종합시장']

In [62]:
df_necessities_sales = df_necessities.merge(df_sales, how='inner', on=['시장이름', '분기', '년도', '업종'])

In [78]:
df_necessities_sales

,일련번호,시장이름,품목이름,실판매규격,가격(원),년도-월,자치구이름,년도,분기,업종,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,1621162,통인시장,양파(1.5kg망),1kg,2000,2020-12,종로구,2020,4,청과상,...,0,8262,13106,57,2253,1803,7079,5032,5144,4
1,1622882,통인시장,양파(1.5kg망),1kg,2000,2020-12,종로구,2020,4,청과상,...,0,8262,13106,57,2253,1803,7079,5032,5144,4
2,1622883,통인시장,상추(100g),400g,3000,2020-12,종로구,2020,4,청과상,...,0,8262,13106,57,2253,1803,7079,5032,5144,4
3,1622885,통인시장,호박(인큐베이터),1개,1000,2020-12,종로구,2020,4,청과상,...,0,8262,13106,57,2253,1803,7079,5032,5144,4
4,1621163,통인시장,상추(100g),400g,2000,2020-12,종로구,2020,4,청과상,...,0,8262,13106,57,2253,1803,7079,5032,5144,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149919,570660,마천시장,"고등어(생물,국산)",1마리(30cm),4980,2014-02,송파구,2014,1,수산물판매,...,674,6178,5801,0,486,1576,3963,3943,2009,11
149920,557858,마천시장,"조기(냉동,국산)",1마리(20cm),2000,2014-01,송파구,2014,1,수산물판매,...,674,6178,5801,0,486,1576,3963,3943,2009,11
149921,557859,마천시장,"명태(러시아,냉동)",1마리(35cm),2400,2014-01,송파구,2014,1,수산물판매,...,674,6178,5801,0,486,1576,3963,3943,2009,11
149922,557860,마천시장,"오징어(냉동,국산)",1마리(25cm),1980,2014-01,송파구,2014,1,수산물판매,...,674,6178,5801,0,486,1576,3963,3943,2009,11


In [65]:
set(df_necessities['시장이름'].unique()) - set(df_necessities_sales.dropna()['시장이름'].unique())

set()

In [66]:
df_sales[(df_sales['시장이름'].str.contains('사당시장'))]['업종'].unique()

array(['한식음식점', '중식음식점', '일반의원', '치과의원', '골프연습장', '미용실', '육류판매', '화장품',
       '가전제품', '의약품', '피부관리실', '유아의류', '슈퍼마켓', '호프-간이주점', '일반의류'],
      dtype=object)

### 마지막 저장

In [67]:
# df_necessities['시장이름'] = df_necessities['시장이름'].replace(market_mapping)
# df_customer['시장이름'] = df_customer['시장이름'].replace(market_mapping)

In [68]:
# df_market_add['시장이름'] = df_market_add['시장이름'].replace(market_mapping2)

In [69]:
# df_necessities['시장이름'] = df_necessities['시장이름'].replace(market_mapping3)
# df_market_add['시장이름'] = df_market_add['시장이름'].replace(market_mapping3)

In [70]:
df_market_add = df_market_add.append({'시장이름' : '도곡시장' , '위도' : 37.49747397256715, '경도' : 127.05162196909993}, ignore_index=True)

In [71]:
df_market_add.to_csv('../datas/huge/df_market_add.csv', encoding='cp949')

In [72]:
# 최종 생필품 데이터 외부저장
df_necessities.to_csv('../datas/huge/df_necessities.csv', encoding='cp949')

In [73]:
# 최종 유동인구 데이터 외부저장, 업데이트
df_customer.to_csv('../datas/huge/df_customer.csv', encoding='cp949')

In [79]:
df_sales.to_csv('../datas/huge/df_sales_after.csv', encoding='cp949')

In [74]:
df_market_add[df_market_add['시장이름'].str.contains('우림')]

,시장이름,위도,경도
244,우림골목시장,37.596658,127.098206
245,우림시장,37.470326,126.920401


In [75]:
df_necessities_market_add = df_necessities.merge(df_market_add, how='left')

In [76]:
df_necessities_market_add[df_necessities_market_add['위도'].isna()]['시장이름'].unique()

array([], dtype=object)